# 🌍 현장에서의 t-test: 올바른 의사결정 가이드

## 📖 이론에서 실무로

지금까지 우리는 놀라운 여행을 했습니다:
- 🌟 완벽한 세상의 정규분포와 Z-검정
- 🍺 1908년 기네스에서 태어난 t-분포
- ⚔️ 상황별 t-test 삼총사
- 🌪️ 현실의 복잡함과 Welch의 해법

이제 **최종 목적지**에 도착했습니다: **실제 세상에서 t-test를 올바르게 사용하기**! 🎯

---

## 🎯 학습 목표

1. **실제 비즈니스 문제** 해결 프로세스
2. **A/B 테스트** 설계와 분석
3. **임상시험** 통계 분석
4. **p-hacking 방지**와 올바른 해석
5. **전체 분석 파이프라인** 구축

In [ ]:
# 필수 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from ipywidgets import interact, widgets
import warnings
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

print(\"🌍 실전 t-test 마스터가 되는 여행의 시작!\")
print(\"🎯 이론에서 실무로, 완전 정복을 목표로!\")

## 💻 1. A/B 테스트: 웹사이트 전환율 개선

### 📱 사례: 모바일 앱 버튼 색상 최적화

**상황**: 
- 전자상거래 앱의 '구매하기' 버튼 색상 테스트
- 기존: 파란색 버튼 (A그룹)
- 신규: 주황색 버튼 (B그룹)
- 목표: 클릭률(CTR) 개선

### 🎯 올바른 A/B 테스트 설계

1. **가설 설정**
   - H₀: 두 버튼의 클릭률이 같다 (p_A = p_B)
   - H₁: 주황색 버튼의 클릭률이 더 높다 (p_B > p_A)

2. **표본 크기 계산**
   - 현재 클릭률: 12%
   - 목표 개선: 15% (상대적 25% 개선)
   - 검정력: 80%, 유의수준: 5%

3. **실험 설계**
   - 랜덤 배정
   - 동시 실행
   - 충분한 실행 기간

In [ ]:
# 💻 A/B 테스트 시뮬레이션
class ABTestAnalyzer:
    def __init__(self):
        self.name = \"A/B 테스트 분석기\"
    
    def simulate_ab_test(self, n_control, n_treatment, p_control, p_treatment, seed=42):
        \"\"\"A/B 테스트 데이터 시뮬레이션\"\"\"
        np.random.seed(seed)
        
        # 클릭 여부 생성 (베르누이 분포)
        control_clicks = np.random.binomial(1, p_control, n_control)
        treatment_clicks = np.random.binomial(1, p_treatment, n_treatment)
        
        return control_clicks, treatment_clicks
    
    def analyze_ab_test(self, control_data, treatment_data, alpha=0.05):
        \"\"\"A/B 테스트 분석\"\"\"
        
        # 기본 통계
        n_control = len(control_data)
        n_treatment = len(treatment_data)
        clicks_control = np.sum(control_data)
        clicks_treatment = np.sum(treatment_data)
        ctr_control = clicks_control / n_control
        ctr_treatment = clicks_treatment / n_treatment
        
        # 이항 비율의 차이 검정 (근사적으로 t-검정 사용)
        # 정확히는 z-검정이지만, 실습을 위해 t-검정으로 변환
        
        # 베르누이 분포의 평균과 분산
        # E[X] = p, Var[X] = p(1-p)
        var_control = ctr_control * (1 - ctr_control)
        var_treatment = ctr_treatment * (1 - ctr_treatment)
        
        # 표준오차 계산
        se = np.sqrt(var_control/n_control + var_treatment/n_treatment)
        
        # t-통계량 (근사)
        t_stat = (ctr_treatment - ctr_control) / se
        
        # 자유도 (Welch-Satterthwaite)
        df = (var_control/n_control + var_treatment/n_treatment)**2 / (
            (var_control/n_control)**2/(n_control-1) + 
            (var_treatment/n_treatment)**2/(n_treatment-1)
        )
        
        # p-값 (단측 검정: treatment > control)
        p_value = 1 - stats.t.cdf(t_stat, df)
        
        # 신뢰구간
        margin_error = stats.t.ppf(1 - alpha/2, df) * se
        diff = ctr_treatment - ctr_control
        ci_lower = diff - margin_error
        ci_upper = diff + margin_error
        
        # 효과크기 (Cohen's h for proportions)
        cohens_h = 2 * (np.arcsin(np.sqrt(ctr_treatment)) - np.arcsin(np.sqrt(ctr_control)))
        
        # 상대적 개선률
        relative_improvement = (ctr_treatment - ctr_control) / ctr_control * 100
        
        return {
            'n_control': n_control,
            'n_treatment': n_treatment,
            'clicks_control': clicks_control,
            'clicks_treatment': clicks_treatment,
            'ctr_control': ctr_control,
            'ctr_treatment': ctr_treatment,
            'difference': diff,
            'relative_improvement': relative_improvement,
            't_statistic': t_stat,
            'df': df,
            'p_value': p_value,
            'significant': p_value < alpha,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'cohens_h': cohens_h,
            'alpha': alpha
        }
    
    def visualize_ab_test(self, result):
        \"\"\"A/B 테스트 결과 시각화\"\"\"
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                '1. 클릭률 비교',
                '2. 신뢰구간',
                '3. 통계적 유의성',
                '4. 비즈니스 임팩트'
            ]
        )
        
        # 1. 클릭률 바 차트
        fig.add_trace(
            go.Bar(
                x=['Control (A)', 'Treatment (B)'],
                y=[result['ctr_control']*100, result['ctr_treatment']*100],
                name='클릭률 (%)',
                text=[f\"{result['ctr_control']*100:.1f}%\", 
                      f\"{result['ctr_treatment']*100:.1f}%\"],
                textposition='auto',
                marker_color=['lightblue', 'orange']
            ),
            row=1, col=1
        )
        
        # 2. 차이의 신뢰구간
        fig.add_trace(
            go.Scatter(
                x=[result['ci_lower']*100, result['ci_upper']*100],
                y=[1, 1],
                mode='lines+markers',
                name='95% 신뢰구간',
                line=dict(color='green', width=5),
                marker=dict(size=10)
            ),
            row=1, col=2
        )
        
        # 차이 점 추가
        fig.add_trace(
            go.Scatter(
                x=[result['difference']*100],
                y=[1.1],
                mode='markers',
                name='관찰된 차이',
                marker=dict(color='red', size=12, symbol='diamond')
            ),
            row=1, col=2
        )
        
        # 차이 없음 선
        fig.add_vline(
            x=0, line_dash=\"dash\", line_color=\"black\",
            annotation_text=\"차이 없음\", row=1, col=2
        )
        
        # 3. p-값 시각화
        significance_text = \"통계적으로 유의\" if result['significant'] else \"통계적으로 무의미\"
        color = 'green' if result['significant'] else 'red'
        
        fig.add_trace(
            go.Bar(
                x=['p-값'],
                y=[result['p_value']],
                name=significance_text,
                text=[f\"{result['p_value']:.4f}\"],
                textposition='auto',
                marker_color=color
            ),
            row=2, col=1
        )
        
        # 유의수준 선
        fig.add_hline(
            y=result['alpha'], line_dash=\"dash\", line_color=\"red\",
            annotation_text=f\"α = {result['alpha']}\", row=2, col=1
        )
        
        # 4. 비즈니스 임팩트
        metrics = ['상대적 개선률 (%)', 'Cohen\'s h']
        values = [result['relative_improvement'], result['cohens_h']]
        
        fig.add_trace(
            go.Bar(
                x=metrics,
                y=values,
                name='효과 크기',
                text=[f\"{v:.2f}\" for v in values],
                textposition='auto',
                marker_color=['lightcoral', 'lightgreen']
            ),
            row=2, col=2
        )
        
        # 레이아웃
        decision = \"B그룹 승리!\" if result['significant'] and result['difference'] > 0 else \"A그룹 유지\"
        fig.update_layout(
            title=f'💻 A/B 테스트 결과: {decision}',
            height=800
        )
        
        return fig

# A/B 테스트 실행
ab_tester = ABTestAnalyzer()

# 시나리오: 주황색 버튼이 실제로 더 좋은 경우
control_data, treatment_data = ab_tester.simulate_ab_test(
    n_control=1000,     # A그룹 (파란색 버튼)
    n_treatment=1000,   # B그룹 (주황색 버튼)
    p_control=0.12,     # 기존 클릭률 12%
    p_treatment=0.15    # 새로운 클릭률 15%
)

# 분석 수행
ab_result = ab_tester.analyze_ab_test(control_data, treatment_data)

# 시각화
fig_ab = ab_tester.visualize_ab_test(ab_result)
fig_ab.show()

# 결과 보고서
print(\"💻 A/B 테스트 최종 보고서\")
print(\"=\" * 50)
print(f\"\n📊 실험 설계:")
print(f\"   A그룹 (Control): {ab_result['n_control']:,}명")
print(f\"   B그룹 (Treatment): {ab_result['n_treatment']:,}명")

print(f\"\n📈 주요 결과:")
print(f\"   A그룹 클릭률: {ab_result['ctr_control']:.1%} ({ab_result['clicks_control']}/{ab_result['n_control']})")
print(f\"   B그룹 클릭률: {ab_result['ctr_treatment']:.1%} ({ab_result['clicks_treatment']}/{ab_result['n_treatment']})")
print(f\"   절대적 차이: {ab_result['difference']:.1%}p")
print(f\"   상대적 개선: {ab_result['relative_improvement']:.1f}%")

print(f\"\n📊 통계 분석:")
print(f\"   t-통계량: {ab_result['t_statistic']:.3f}")
print(f\"   p-값: {ab_result['p_value']:.4f}")
print(f\"   95% 신뢰구간: [{ab_result['ci_lower']:.1%}, {ab_result['ci_upper']:.1%}]")

print(f\"\n🎯 결론:")
if ab_result['significant']:
    print(f\"   ✅ 주황색 버튼이 통계적으로 유의하게 더 효과적입니다!")
    print(f\"   💰 예상 수익 증대: {ab_result['relative_improvement']:.1f}%")
    print(f\"   📋 권장사항: B그룹(주황색) 버튼으로 전환")
else:
    print(f\"   ❌ 통계적으로 유의한 차이가 없습니다.")
    print(f\"   📋 권장사항: 추가 실험 또는 다른 요소 테스트")

print(f\"\n💡 효과크기 해석:")
if abs(ab_result['cohens_h']) < 0.2:
    effect_interpretation = \"매우 작은 효과\"
elif abs(ab_result['cohens_h']) < 0.5:
    effect_interpretation = \"작은 효과\"
elif abs(ab_result['cohens_h']) < 0.8:
    effect_interpretation = \"중간 효과\"
else:
    effect_interpretation = \"큰 효과\"
    
print(f\"   Cohen's h = {ab_result['cohens_h']:.3f} ({effect_interpretation})")

## 🏥 2. 임상시험: 신약 효과 검증

### 💊 사례: 고혈압 치료제 임상시험

**연구 설계**:
- 대상: 고혈압 환자 60명
- 방법: 무작위 대조 임상시험 (RCT)
- 평가: 8주 치료 후 수축기 혈압 변화
- 주요 평가변수: 치료 전후 혈압 차이

### 🎯 임상시험에서의 t-검정

1. **연구 가설**
   - H₀: 신약의 혈압 강하 효과가 없다 (μ_diff = 0)
   - H₁: 신약이 혈압을 유의하게 낮춘다 (μ_diff < 0)

2. **통계 계획**
   - 유의수준: α = 0.05
   - 검정력: 80%
   - 효과크기: 10mmHg 감소 (임상적 의미)

In [ ]:
# 🏥 임상시험 분석
class ClinicalTrialAnalyzer:
    def __init__(self):
        self.name = \"임상시험 분석기\"
    
    def simulate_clinical_trial(self, n_patients, true_effect, baseline_mean, baseline_std, seed=42):
        \"\"\"임상시험 데이터 시뮬레이션\"\"\"
        np.random.seed(seed)
        
        # 환자들의 초기 혈압 (개인차 반영)
        baseline_bp = np.random.normal(baseline_mean, baseline_std, n_patients)
        
        # 치료 효과 (개인차와 측정 오차 포함)
        treatment_effect = np.random.normal(true_effect, 5, n_patients)
        
        # 치료 후 혈압
        followup_bp = baseline_bp + treatment_effect
        
        return baseline_bp, followup_bp
    
    def analyze_clinical_trial(self, baseline, followup, alpha=0.05):
        \"\"\"임상시험 통계 분석\"\"\"
        
        # 변화량 계산
        change = followup - baseline
        
        # 기본 통계
        n = len(change)
        mean_baseline = np.mean(baseline)
        mean_followup = np.mean(followup)
        mean_change = np.mean(change)
        std_change = np.std(change, ddof=1)
        se_change = std_change / np.sqrt(n)
        
        # One-sample t-test (변화량이 0과 다른가?)
        t_stat = mean_change / se_change
        df = n - 1
        
        # p-값 (단측 검정: 혈압이 감소했는가?)
        p_value_one_sided = stats.t.cdf(t_stat, df)  # 음수 방향
        p_value_two_sided = 2 * min(p_value_one_sided, 1 - p_value_one_sided)
        
        # 신뢰구간
        margin_error = stats.t.ppf(1 - alpha/2, df) * se_change
        ci_lower = mean_change - margin_error
        ci_upper = mean_change + margin_error
        
        # 효과크기 (Cohen's d)
        cohens_d = mean_change / std_change
        
        # 임상적 의미 (10mmHg 이상 감소)
        clinically_meaningful = mean_change <= -10
        
        # 응답률 (개별 환자 기준)
        responders = np.sum(change <= -10)  # 10mmHg 이상 감소한 환자 수
        response_rate = responders / n
        
        return {
            'n_patients': n,
            'mean_baseline': mean_baseline,
            'mean_followup': mean_followup,
            'mean_change': mean_change,
            'std_change': std_change,
            'se_change': se_change,
            't_statistic': t_stat,
            'df': df,
            'p_value_one_sided': p_value_one_sided,
            'p_value_two_sided': p_value_two_sided,
            'significant_one_sided': p_value_one_sided < alpha,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'cohens_d': cohens_d,
            'clinically_meaningful': clinically_meaningful,
            'responders': responders,
            'response_rate': response_rate,
            'change_data': change
        }
    
    def create_clinical_report(self, result):
        \"\"\"임상시험 보고서 생성\"\"\"
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                '1. 혈압 변화 분포',
                '2. 치료 전후 비교',
                '3. 신뢰구간',
                '4. 응답률 분석'
            ],
            specs=[
                [{'type': 'histogram'}, {'type': 'scatter'}],
                [{'type': 'scatter'}, {'type': 'bar'}]
            ]
        )
        
        # 1. 혈압 변화량 히스토그램
        fig.add_trace(
            go.Histogram(
                x=result['change_data'],
                nbinsx=15,
                name='혈압 변화량',
                marker_color='lightblue',
                opacity=0.7
            ),
            row=1, col=1
        )
        
        # 평균 변화량 선
        fig.add_vline(
            x=result['mean_change'],
            line_dash=\"solid\", line_color=\"red\",
            annotation_text=f\"평균: {result['mean_change']:.1f}mmHg\",
            row=1, col=1
        )
        
        # 임상적 의미 기준선
        fig.add_vline(
            x=-10, line_dash=\"dash\", line_color=\"green\",
            annotation_text=\"임상적 의미 (-10mmHg)\", row=1, col=1
        )
        
        # 2. 치료 전후 비교 (개별 환자)
        patient_ids = list(range(1, min(21, result['n_patients']+1)))  # 처음 20명만
        baseline_subset = [result['mean_baseline']] * len(patient_ids)  # 간소화
        followup_subset = [result['mean_baseline'] + result['mean_change']] * len(patient_ids)
        
        for i, (before, after) in enumerate(zip(baseline_subset, followup_subset)):
            fig.add_trace(
                go.Scatter(
                    x=['치료 전', '치료 후'],
                    y=[before, after],
                    mode='lines+markers',
                    line=dict(width=1, color='lightgray'),
                    marker=dict(size=4),
                    showlegend=False
                ),
                row=1, col=2
            )
        
        # 평균 변화 강조
        fig.add_trace(
            go.Scatter(
                x=['치료 전', '치료 후'],
                y=[result['mean_baseline'], result['mean_followup']],
                mode='lines+markers',
                line=dict(width=4, color='red'),
                marker=dict(size=10),
                name='평균 변화'
            ),
            row=1, col=2
        )
        
        # 3. 신뢰구간
        fig.add_trace(
            go.Scatter(
                x=[result['ci_lower'], result['ci_upper']],
                y=[1, 1],
                mode='lines+markers',
                name='95% 신뢰구간',
                line=dict(color='green', width=5),
                marker=dict(size=10)
            ),
            row=2, col=1
        )
        
        # 평균 변화량
        fig.add_trace(
            go.Scatter(
                x=[result['mean_change']],
                y=[1.1],
                mode='markers',
                name='평균 변화량',
                marker=dict(color='red', size=12, symbol='diamond')
            ),
            row=2, col=1
        )
        
        # 4. 응답률 분석
        response_categories = ['응답자', '무응답자']
        response_counts = [result['responders'], result['n_patients'] - result['responders']]
        
        fig.add_trace(
            go.Bar(
                x=response_categories,
                y=response_counts,
                name='환자 수',
                text=[f\"{count}명 ({count/result['n_patients']:.1%})\" for count in response_counts],
                textposition='auto',
                marker_color=['green', 'lightgray']
            ),
            row=2, col=2
        )
        
        # 레이아웃
        efficacy = \"효과적\" if result['significant_one_sided'] else \"효과 불명확\"
        fig.update_layout(
            title=f'🏥 임상시험 결과: 신약은 {efficacy}',
            height=800
        )
        
        return fig

# 임상시험 시뮬레이션
clinical_analyzer = ClinicalTrialAnalyzer()

# 데이터 생성 (평균 12mmHg 감소 효과)
baseline_bp, followup_bp = clinical_analyzer.simulate_clinical_trial(
    n_patients=60,
    true_effect=-12,      # 평균 12mmHg 감소
    baseline_mean=150,    # 초기 수축기 혈압 150mmHg
    baseline_std=15       # 개인차
)

# 분석 수행
clinical_result = clinical_analyzer.analyze_clinical_trial(baseline_bp, followup_bp)

# 시각화
fig_clinical = clinical_analyzer.create_clinical_report(clinical_result)
fig_clinical.show()

# 임상시험 보고서
print(\"🏥 임상시험 최종 보고서\")
print(\"=\" * 50)
print(f\"\n📋 연구 개요:")
print(f\"   대상 환자: {clinical_result['n_patients']}명")
print(f\"   연구 설계: 단일군, 전후 비교")
print(f\"   주요 평가변수: 수축기 혈압 변화")

print(f\"\n📊 주요 결과:")
print(f\"   치료 전 평균: {clinical_result['mean_baseline']:.1f} ± {np.std(baseline_bp, ddof=1):.1f} mmHg")
print(f\"   치료 후 평균: {clinical_result['mean_followup']:.1f} ± {np.std(followup_bp, ddof=1):.1f} mmHg")
print(f\"   평균 변화량: {clinical_result['mean_change']:.1f} ± {clinical_result['std_change']:.1f} mmHg")

print(f\"\n📈 통계 분석:")
print(f\"   t-통계량: {clinical_result['t_statistic']:.3f}")
print(f\"   p-값 (단측): {clinical_result['p_value_one_sided']:.4f}")
print(f\"   95% 신뢰구간: [{clinical_result['ci_lower']:.1f}, {clinical_result['ci_upper']:.1f}] mmHg")
print(f\"   효과크기 (Cohen's d): {clinical_result['cohens_d']:.3f}")

print(f\"\n🎯 임상적 평가:")
print(f\"   통계적 유의성: {'예' if clinical_result['significant_one_sided'] else '아니오'} (p {'<' if clinical_result['significant_one_sided'] else '≥'} 0.05)")
print(f\"   임상적 의미: {'예' if clinical_result['clinically_meaningful'] else '아니오'} (≥10mmHg 감소)")
print(f\"   응답률: {clinical_result['response_rate']:.1%} ({clinical_result['responders']}/{clinical_result['n_patients']}명)")

print(f\"\n📋 결론:")
if clinical_result['significant_one_sided'] and clinical_result['clinically_meaningful']:
    print(f\"   ✅ 신약은 통계적으로 유의하고 임상적으로 의미있는 혈압 강하 효과를 보였습니다.")
    print(f\"   💊 권장사항: 다음 단계 임상시험 진행")
elif clinical_result['significant_one_sided']:
    print(f\"   ⚡ 통계적으로 유의하지만 임상적 의미는 제한적입니다.")
    print(f\"   💊 권장사항: 용량 조정 또는 추가 연구 필요")
else:
    print(f\"   ❌ 통계적으로 유의한 효과가 관찰되지 않았습니다.")
    print(f\"   💊 권장사항: 연구 중단 또는 설계 변경")

## ⚠️ 3. p-hacking 방지와 올바른 해석

### 🚨 p-hacking의 위험성

**p-hacking**은 통계적 유의성을 얻기 위해 부적절한 방법을 사용하는 것입니다:

1. **다중 비교 문제**: 여러 번 검정하면서 유의한 결과만 보고
2. **선택적 보고**: 원하는 결과가 나올 때까지 분석 반복
3. **사후 가설**: 데이터를 본 후 가설 설정
4. **체리피킹**: 일부 데이터만 선택적으로 사용

### ✅ 올바른 통계 실무

1. **사전 계획**
   - 가설과 분석 계획을 데이터 수집 전에 수립
   - 표본 크기 계산
   - 주요/부차 평가변수 명시

2. **다중 비교 보정**
   - Bonferroni 보정
   - FDR (False Discovery Rate) 제어
   - 계획된 비교만 수행

3. **효과크기 보고**
   - p-값과 함께 효과크기 보고
   - 신뢰구간 제시
   - 실용적 의미 해석

4. **재현 가능성**
   - 코드와 데이터 공개
   - 분석 과정 문서화
   - 독립적 검증

In [ ]:
# ⚠️ p-hacking 시뮬레이션과 올바른 방법
def demonstrate_p_hacking():
    \"\"\"p-hacking의 문제점을 시연\"\"\"
    
    np.random.seed(42)
    n_tests = 20  # 20번의 다른 비교
    p_values = []
    
    print(\"🚨 p-hacking 시뮬레이션: 실제로는 차이가 없는 상황\")
    print(\"="*60)
    
    # 실제로는 차이가 없는 두 그룹을 여러 번 비교
    for i in range(n_tests):
        # 동일한 분포에서 두 그룹 생성 (실제 차이 없음)
        group1 = np.random.normal(100, 15, 30)
        group2 = np.random.normal(100, 15, 30)
        
        # t-검정 수행
        t_stat, p_val = stats.ttest_ind(group1, group2)
        p_values.append(p_val)
        
        status = \"🎉 유의!\" if p_val < 0.05 else \"무의미\"
        print(f\"   비교 {i+1:2d}: p = {p_val:.4f} {status}")
    
    # 결과 분석
    significant_count = sum(1 for p in p_values if p < 0.05)
    
    print(f\"\n📊 결과 요약:")
    print(f\"   총 비교 횟수: {n_tests}")
    print(f\"   유의한 결과: {significant_count}개 ({significant_count/n_tests:.1%})")
    print(f\"   예상 Type I 오류: ~{n_tests*0.05:.0f}개 (5%)")
    
    if significant_count > 0:
        print(f\"\n⚠️ 경고: 차이가 없는데도 {significant_count}개의 '유의한' 결과가 나왔습니다!")
        print(f\"💡 이것이 바로 p-hacking의 위험성입니다.")
    
    return p_values

def correct_multiple_testing(p_values, method='bonferroni'):
    \"\"\"다중 검정 보정\"\"\"
    
    if method == 'bonferroni':
        # Bonferroni 보정
        corrected_alpha = 0.05 / len(p_values)
        significant_after_correction = [p < corrected_alpha for p in p_values]
        
        print(f\"\n🛡️ Bonferroni 보정 결과:")
        print(f\"   보정된 α: {corrected_alpha:.4f}")
        print(f\"   보정 후 유의한 결과: {sum(significant_after_correction)}개")
        
    elif method == 'fdr':
        # False Discovery Rate (Benjamini-Hochberg)
        from statsmodels.stats.multitest import multipletests
        reject, corrected_p, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
        
        print(f\"\n🎯 FDR 보정 결과:")
        print(f\"   보정 후 유의한 결과: {sum(reject)}개")
        print(f\"   FDR < 0.05로 제어됨")
        
        return reject, corrected_p
    
    return significant_after_correction

# p-hacking 시연
p_vals = demonstrate_p_hacking()

# 올바른 다중 검정 보정
bonferroni_results = correct_multiple_testing(p_vals, 'bonferroni')

print(f\"\n💡 올바른 통계 실무 가이드라인:\")
print(f\"\n1. 🎯 사전 계획:\n   - 주가설을 미리 정의\n   - 표본 크기 사전 계산\n   - 분석 계획서 작성")
print(f\"\n2. 🛡️ 다중 비교 대응:\n   - Bonferroni 보정 (보수적)\n   - FDR 제어 (균형적)\n   - 계획된 비교만 수행")
print(f\"\n3. 📊 종합 보고:\n   - p-값 + 효과크기\n   - 신뢰구간 포함\n   - 실용적 의미 해석")
print(f\"\n4. 🔍 투명성:\n   - 모든 분석 과정 공개\n   - 실패한 실험도 보고\n   - 재현 가능한 코드")

# 올바른 분석 예시
print(f\"\n\n✅ 올바른 분석 예시:\")
print(f\"="*40)

# 단일 계획된 비교
np.random.seed(123)
control_group = np.random.normal(100, 15, 50)
treatment_group = np.random.normal(108, 15, 50)  # 실제 8점 차이

# 사전 계획된 t-검정
t_stat, p_val = stats.ttest_ind(treatment_group, control_group)
effect_size = (np.mean(treatment_group) - np.mean(control_group)) / np.sqrt(((49*np.var(treatment_group, ddof=1) + 49*np.var(control_group, ddof=1))/98))

print(f\"📋 사전 계획된 주 가설 검정:")
print(f\"   가설: 치료군이 대조군보다 높은 점수를 가질 것이다")
print(f\"   대조군 평균: {np.mean(control_group):.1f}")
print(f\"   치료군 평균: {np.mean(treatment_group):.1f}")
print(f\"   t-통계량: {t_stat:.3f}")
print(f\"   p-값: {p_val:.4f}")
print(f\"   효과크기: {effect_size:.3f}")
print(f\"   결론: {'유의한 효과' if p_val < 0.05 else '효과 없음'}")
print(f\"   해석: {'실용적으로 의미있는 개선' if effect_size > 0.5 else '효과가 있지만 작음'}")

## 🔧 4. 전체 분석 파이프라인

### 📋 체크리스트: 완벽한 t-검정 분석

#### 1단계: 계획 수립 📊
- [ ] 연구 질문 명확화
- [ ] 가설 설정 (H₀, H₁)
- [ ] 적절한 t-검정 유형 선택
- [ ] 표본 크기 계산
- [ ] α 수준 설정 (보통 0.05)

#### 2단계: 데이터 수집 📈
- [ ] 무작위 표본 추출
- [ ] 독립성 확보
- [ ] 충분한 표본 크기
- [ ] 결측치 처리 계획

#### 3단계: 가정 확인 🔍
- [ ] 정규성 검정 (Shapiro-Wilk)
- [ ] 등분산성 검정 (Levene's test)
- [ ] 이상치 탐지
- [ ] 독립성 확인

#### 4단계: 분석 실행 ⚡
- [ ] 적절한 t-검정 수행
- [ ] 신뢰구간 계산
- [ ] 효과크기 계산
- [ ] 다중 비교 보정 (필요시)

#### 5단계: 결과 해석 💡
- [ ] 통계적 유의성 평가
- [ ] 실용적 의미 평가
- [ ] 신뢰구간 해석
- [ ] 한계점 인정

#### 6단계: 보고 📝
- [ ] 투명한 방법론 기술
- [ ] 완전한 결과 보고
- [ ] 시각화 포함
- [ ] 재현 가능한 코드

In [ ]:
# 🔧 완전한 분석 파이프라인 구현
class CompleteTTestPipeline:
    def __init__(self):
        self.analysis_log = []
        
    def log(self, message):
        \"\"\"분석 과정 로깅\"\"\"
        self.analysis_log.append(message)
        print(f\"📝 {message}\")
    
    def complete_analysis(self, group1, group2, group1_name='그룹 1', group2_name='그룹 2', 
                         alpha=0.05, research_question=None):
        \"\"\"완전한 t-검정 분석 파이프라인\"\"\"
        
        self.log(\"=== t-검정 분석 파이프라인 시작 ===\")
        
        if research_question:
            self.log(f\"연구 질문: {research_question}\")
        
        # 1단계: 기초 통계
        self.log(\"1단계: 기초 통계 계산\")
        n1, n2 = len(group1), len(group2)
        mean1, mean2 = np.mean(group1), np.mean(group2)
        std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
        
        self.log(f\"  {group1_name}: n={n1}, 평균={mean1:.2f}, SD={std1:.2f}\")
        self.log(f\"  {group2_name}: n={n2}, 평균={mean2:.2f}, SD={std2:.2f}\")
        
        # 2단계: 가정 확인
        self.log(\"2단계: t-검정 가정 확인\")
        
        # 정규성 검정
        shapiro1 = stats.shapiro(group1)
        shapiro2 = stats.shapiro(group2)
        normality_ok = shapiro1[1] > 0.05 and shapiro2[1] > 0.05
        
        self.log(f\"  정규성 ({group1_name}): p={shapiro1[1]:.4f} {'✅' if shapiro1[1] > 0.05 else '❌'}\")
        self.log(f\"  정규성 ({group2_name}): p={shapiro2[1]:.4f} {'✅' if shapiro2[1] > 0.05 else '❌'}\")
        
        # 등분산성 검정
        levene_stat, levene_p = stats.levene(group1, group2)
        equal_var = levene_p > 0.05
        
        self.log(f\"  등분산성 (Levene): p={levene_p:.4f} {'✅' if equal_var else '❌'}\")
        
        # 분산 비율
        var_ratio = max(std1**2, std2**2) / min(std1**2, std2**2)
        self.log(f\"  분산 비율: {var_ratio:.2f} {'✅' if var_ratio < 4 else '⚠️ 큰 차이'}\")
        
        # 3단계: 적절한 검정 선택
        self.log(\"3단계: 검정 방법 선택\")
        
        if not normality_ok and min(n1, n2) < 30:
            self.log(\"  권장: Mann-Whitney U 검정 (정규성 위반, 작은 표본)\")
            test_method = \"mann-whitney\"
        elif not equal_var or var_ratio > 4:
            self.log(\"  선택: Welch's t-test (등분산 가정 위반)\")
            test_method = \"welch\"
        else:
            self.log(\"  선택: Student's t-test (모든 가정 만족)\")
            test_method = \"student\"
        
        # 4단계: 검정 수행
        self.log(\"4단계: 통계 검정 수행\")
        
        if test_method == \"mann-whitney\":
            # Mann-Whitney U 검정
            u_stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')
            self.log(f\"  Mann-Whitney U: U={u_stat:.3f}, p={p_value:.4f}\")
            
            # 효과크기 근사
            effect_size = None
            ci_lower, ci_upper = None, None
            
        else:
            # t-검정
            use_equal_var = (test_method == \"student\")
            t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=use_equal_var)
            
            self.log(f\"  {test_method.title()}: t={t_stat:.3f}, p={p_value:.4f}\")
            
            # 효과크기 (Cohen's d)
            if use_equal_var:
                pooled_std = np.sqrt(((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2))
                effect_size = (mean1 - mean2) / pooled_std
            else:
                effect_size = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
            
            # 신뢰구간 (근사)
            se = np.sqrt(std1**2/n1 + std2**2/n2)
            df = n1 + n2 - 2 if use_equal_var else (std1**2/n1 + std2**2/n2)**2 / ((std1**2/n1)**2/(n1-1) + (std2**2/n2)**2/(n2-1))
            margin = stats.t.ppf(1-alpha/2, df) * se
            diff = mean1 - mean2
            ci_lower, ci_upper = diff - margin, diff + margin
        
        # 5단계: 결과 해석
        self.log(\"5단계: 결과 해석\")
        
        significant = p_value < alpha
        self.log(f\"  통계적 유의성: {'유의함' if significant else '유의하지 않음'} (p {'<' if significant else '≥'} {alpha})\")
        
        if effect_size is not None:
            if abs(effect_size) < 0.2:
                effect_interp = \"매우 작은 효과\"
            elif abs(effect_size) < 0.5:
                effect_interp = \"작은 효과\"
            elif abs(effect_size) < 0.8:
                effect_interp = \"중간 효과\"
            else:
                effect_interp = \"큰 효과\"
            
            self.log(f\"  효과크기: d={effect_size:.3f} ({effect_interp})\")
        
        if ci_lower is not None and ci_upper is not None:
            self.log(f\"  95% 신뢰구간: [{ci_lower:.3f}, {ci_upper:.3f}]\")
        
        # 6단계: 최종 결론
        self.log(\"6단계: 최종 결론\")
        
        if significant and effect_size is not None and abs(effect_size) >= 0.5:
            conclusion = \"통계적으로 유의하고 실용적으로 의미있는 차이가 있습니다.\"
        elif significant:
            conclusion = \"통계적으로 유의하지만 효과크기가 작습니다.\"
        else:
            conclusion = \"통계적으로 유의한 차이가 없습니다.\"
        
        self.log(f\"  결론: {conclusion}\")
        
        # 권장사항
        if not normality_ok:
            self.log(\"  💡 권장: 정규성 위반으로 인해 비모수 검정 결과도 함께 고려하세요.\")
        if not equal_var:
            self.log(\"  💡 권장: 등분산 가정 위반으로 Welch's t-test 결과를 신뢰하세요.\")
        if var_ratio > 10:
            self.log(\"  ⚠️ 경고: 분산 차이가 매우 큽니다. 데이터 변환을 고려하세요.\")
        
        self.log(\"=== 분석 완료 ===\")
        
        return {
            'test_method': test_method,
            'statistic': t_stat if test_method != \"mann-whitney\" else u_stat,
            'p_value': p_value,
            'significant': significant,
            'effect_size': effect_size,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'conclusion': conclusion,
            'assumptions': {
                'normality': normality_ok,
                'equal_variance': equal_var,
                'variance_ratio': var_ratio
            }
        }

# 완전한 분석 파이프라인 실행
pipeline = CompleteTTestPipeline()

# 예제 데이터: 두 교육 방법 비교
np.random.seed(2023)
traditional_method = np.random.normal(75, 12, 35)  # 전통적 방법
new_method = np.random.normal(82, 15, 32)          # 새로운 방법

print(\"\n🎓 종합 사례: 교육 방법 효과 비교\")
print(\"="*60)

# 완전한 분석 수행
result = pipeline.complete_analysis(
    traditional_method, new_method,
    '전통적 방법', '새로운 방법',
    research_question=\"새로운 교육 방법이 전통적 방법보다 효과적인가?\"
)

print(f\"\n📋 분석 요약:")
print(f\"   검정 방법: {result['test_method']}")
print(f\"   통계량: {result['statistic']:.3f}")
print(f\"   p-값: {result['p_value']:.4f}")
print(f\"   효과크기: {result['effect_size']:.3f if result['effect_size'] else 'N/A'}")
print(f\"   최종 결론: {result['conclusion']}")

## 🎉 여행의 끝: 1908년부터 현재까지

### 🌟 우리가 지나온 길

1. **🌟 완벽한 세상** (Z-검정)
   - 정규분포의 아름다움
   - 중심극한정리의 마법
   - σ를 아는 이상적 세계

2. **🍺 현실의 발견** (t-분포)
   - Gosset의 기네스 맥주 고민
   - 작은 표본의 불확실성
   - Student's t-분포의 탄생

3. **⚔️ 삼총사의 활약** (3가지 t-test)
   - 아토스: One-sample t-test
   - 포르토스: Independent t-test  
   - 아라미스: Paired t-test

4. **🌪️ 현실의 복잡함** (가정 위반)
   - Welch의 혁신적 해법
   - Satterthwaite 자유도
   - 강건한 통계학

5. **🌍 실전 적용** (올바른 실무)
   - A/B 테스트 설계
   - 임상시험 분석
   - p-hacking 방지
   - 완전한 분석 파이프라인

### 🎯 핵심 교훈

1. **📊 통계는 도구일 뿐**
   - p-값 < 0.05가 전부가 아님
   - 효과크기와 실용적 의미가 중요
   - 맥락과 도메인 지식 필수

2. **🛡️ 가정보다는 강건성**
   - 완벽한 가정은 현실에 없음
   - Welch's t-test가 더 안전한 선택
   - 의심스러우면 검증하기

3. **🔍 투명성과 재현성**
   - 모든 분석 과정 문서화
   - 사전 계획의 중요성
   - 실패한 실험도 가치있음

4. **💡 지속적 학습**
   - 통계학은 진화하는 분야
   - 새로운 방법론 지속 학습
   - 실무와 이론의 균형

### 🚀 다음 단계

t-검정을 마스터한 여러분은 이제:
- **ANOVA**: 3개 이상 그룹 비교
- **회귀분석**: 연속 변수 관계 분석
- **베이지안 통계**: 불확실성의 새로운 패러다임
- **머신러닝**: 예측과 분류의 세계

로 나아갈 준비가 되었습니다!

---

## 📚 마지막 정리

### ⭐ 황금 규칙들

1. **계획 → 수집 → 분석 → 해석** 순서 지키기
2. **가정 확인**은 선택이 아닌 필수
3. **Welch's t-test**를 기본으로 사용
4. **효과크기**와 **신뢰구간** 항상 보고
5. **p-hacking** 절대 금지

### 🎉 축하합니다!

> **"1908년 기네스 맥주공장에서 시작된 여행이  
> 2024년 여러분의 데이터 분석 실력으로 완성되었습니다!"** 🍺✨

이제 여러분은 **진정한 t-test 마스터**입니다! 🏆